## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-06-08-50-56 +0000,wapo,Judge deals Trump new setback in plans to depl...,https://www.washingtonpost.com/nation/2025/10/...
1,2025-10-06-08-46-28 +0000,bbc,NZ compensates Samoa over crashed naval ship,https://www.bbc.com/news/articles/czx0z76nxggo...
2,2025-10-06-08-45-52 +0000,bbc,French Prime Minister Sébastien Lecornu resign...,https://www.bbc.com/news/articles/cewn9k0w9rxo...
3,2025-10-06-08-42-16 +0000,nyt,"Drones, Democracy and the War in Ukraine",https://www.nytimes.com/2025/10/05/world/ukain...
4,2025-10-06-08-39-54 +0000,bbc,Judge blocks Trump from sending troops from Ca...,https://www.bbc.com/news/articles/c740elm70z7o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2297/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
2,trump,36
3,new,10
87,gaza,10
39,shutdown,9
40,guard,9


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
157,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...,139
127,2025-10-05-18-37-30 +0000,latimes,Judge blocks Trump from deploying California N...,https://www.latimes.com/world-nation/story/202...,92
20,2025-10-06-07-17-48 +0000,nypost,Federal judge blocks Trump’s National Guard de...,https://nypost.com/2025/10/06/us-news/federal-...,82
137,2025-10-05-17-22-31 +0000,nypost,Kristi Noem slams ‘delusional’ Chicago mayor w...,https://nypost.com/2025/10/05/us-news/kristi-n...,80
172,2025-10-05-12-15-58 +0000,nyt,Judge Blocks Trump’s Deployment of National Gu...,https://www.nytimes.com/2025/10/04/us/politics...,80


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
157,139,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...
9,40,2025-10-06-08-21-29 +0000,wapo,French prime minister resigns hours after nami...,https://www.washingtonpost.com/world/2025/10/0...
24,37,2025-10-06-06-52-43 +0000,nypost,California biotech tycoon found guilty in murd...,https://nypost.com/2025/10/06/us-news/californ...
68,30,2025-10-05-23-53-33 +0000,bbc,Trump urges mediators to 'move fast' as key Ga...,https://www.bbc.com/news/articles/cj3y6g43248o...
75,28,2025-10-05-23-07-07 +0000,bbc,Gisèle Pelicot to face one of her rapists in c...,https://www.bbc.com/news/articles/cdx2d9lynk1o...
80,27,2025-10-05-22-51-18 +0000,nypost,Chicago police ordered to ignore border patrol...,https://nypost.com/2025/10/05/us-news/chicago-...
156,27,2025-10-05-14-45-55 +0000,nypost,Virginia Beach Dems back Virginia AG nominee J...,https://nypost.com/2025/10/05/us-news/dems-bac...
161,24,2025-10-05-14-17-52 +0000,wapo,Russian air attack across Ukraine damages ener...,https://www.washingtonpost.com/world/2025/10/0...
140,24,2025-10-05-16-44-45 +0000,bbc,Young Tories rate Kemi Badenoch's first year a...,https://www.bbc.com/news/videos/c2375x5endlo?a...
63,23,2025-10-06-00-08-30 +0000,nypost,LI students designed a ‘buddy bot’ — now they’...,https://nypost.com/2025/10/05/us-news/li-stude...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
